# [Day2-3] 파일 I/O와 직렬화 (Serialization)

`학습 목표`

1. `파일 I/O 마스터`: 텍스트와 바이너리 파일을 안전하게 읽고 쓰는 방법을 익힙니다.
2. `Context Manager`: `with` 문을 사용하여 자원을 안전하게 관리합니다.
3. `JSON 핸들링`: API 응답, 설정 파일 등 JSON 데이터를 능숙하게 다룹니다.
4. `데이터 직렬화`: Python 객체를 파일로 저장하고 불러오는 방법을 학습합니다.

---

---

## Part 1: 파일 I/O 기초

파일을 안전하게 읽고 쓰는 방법과 Context Manager를 학습합니다.

---

## 1. [File I/O] 파일 읽기와 쓰기 기본

### 1.1 텍스트 파일 읽기

파일을 열 때는 **모드(mode)**를 지정합니다:
* `'r'` - 읽기 (Read, 기본값)
* `'w'` - 쓰기 (Write, 파일이 있으면 덮어씀)
* `'a'` - 추가 (Append, 파일 끝에 추가)
* `'x'` - 배타적 생성 (파일이 없을 때만 생성)
* `'b'` - 바이너리 모드 (예: `'rb'`, `'wb'`)
* `'+'` - 읽기+쓰기 (예: `'r+'`, `'w+'`)

In [ ]:
# [기본] 파일 전체 읽기
# 먼저 샘플 파일 생성
with open('sample.txt', 'w', encoding='utf-8') as f:
    f.write('안녕하세요\n')
    f.write('데이터 분석 교육입니다\n')
    f.write('Python을 배워봅시다')

# 파일 읽기 - 방법 1: read()
with open('sample.txt', 'r', encoding='utf-8') as f:
    content = f.read()  # 전체 내용을 문자열로
    print(content)

In [ ]:
# 파일 읽기 - 방법 2: readlines()
with open('sample.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()  # 각 줄을 리스트로
    print(lines)
    print(f"총 {len(lines)}줄")

In [ ]:
# 파일 읽기 - 방법 3: 한 줄씩 순회 (메모리 효율적!)
with open('sample.txt', 'r', encoding='utf-8') as f:
    for line_num, line in enumerate(f, 1):
        print(f"{line_num}줄: {line.strip()}")

### 1.2 텍스트 파일 쓰기

In [ ]:
# [실습 1] 쓰기 모드 ('w') - 덮어쓰기
data = ["Apple", "Banana", "Cherry"]

with open('fruits.txt', 'w', encoding='utf-8') as f:
    for fruit in data:
        f.write(fruit + '\n')  # 줄바꿈 추가

print("파일 작성 완료!")

In [ ]:
# [실습 2] 추가 모드 ('a') - 파일 끝에 추가
with open('fruits.txt', 'a', encoding='utf-8') as f:
    f.write('Mango\n')
    f.write('Grape\n')

# 확인
with open('fruits.txt', 'r', encoding='utf-8') as f:
    print(f.read())

---

## 2. [Context Manager] with 문: 안전한 리소스 관리

### 2.1 왜 with 문을 사용하나?

**문제**: `open()`으로 연 파일은 반드시 `close()`로 닫아야 합니다.
* 파일을 닫지 않으면 메모리 누수, 파일 잠금 문제 발생
* 예외가 발생하면 `close()`가 실행되지 않을 수 있음

**해결**: `with` 문은 블록을 벗어나면 **자동으로** 파일을 닫아줍니다!

In [ ]:
# [나쁜 예] 수동으로 닫기 (권장하지 않음)
f = open('example.txt', 'w', encoding='utf-8')
f.write('Hello World')
f.close()  # 잊어버리면 문제 발생!

In [ ]:
# [좋은 예] with 문 사용 (권장)
with open('example.txt', 'w', encoding='utf-8') as f:
    f.write('Hello World')
# 블록을 벗어나면 자동으로 close() 호출됨!

# 파일이 자동으로 닫혔는지 확인
# print(f.closed)  # True

### 2.2 예외 발생 시에도 안전

In [ ]:
# [실습] 예외가 발생해도 파일은 안전하게 닫힘
try:
    with open('test.txt', 'w', encoding='utf-8') as f:
        f.write('안전한 작업\n')
        # raise ValueError("의도적 에러!")  # 예외 발생
        f.write('이 줄도 작성됨\n')
except ValueError as e:
    print(f"에러 발생: {e}")

# 파일은 이미 닫혔으므로 안전!

### 2.3 여러 파일 동시에 열기

In [ ]:
# [실습] 두 파일을 동시에 열어 복사하기
with open('source.txt', 'w', encoding='utf-8') as f:
    f.write('원본 데이터\n복사할 내용')

with open('source.txt', 'r', encoding='utf-8') as src, \
     open('destination.txt', 'w', encoding='utf-8') as dst:
    content = src.read()
    dst.write(content)
    print("파일 복사 완료!")

---

## Part 2: 데이터 직렬화

JSON과 Pickle을 활용한 데이터 저장 및 교환 방법을 학습합니다.

---

---

## 3. [JSON] 데이터 교환의 표준 형식

### 3.1 JSON이란?

**JSON (JavaScript Object Notation)**: 데이터를 저장하고 전송하는 경량 형식
* API 응답, 설정 파일, 데이터 교환 등에 널리 사용
* Python의 딕셔너리/리스트와 거의 동일한 구조
* 텍스트 형식이므로 사람이 읽을 수 있음

**주요 함수**:
* `json.dump(obj, file)` - Python 객체를 파일에 JSON으로 저장
* `json.dumps(obj)` - Python 객체를 JSON 문자열로 변환
* `json.load(file)` - 파일에서 JSON을 읽어 Python 객체로 변환
* `json.loads(string)` - JSON 문자열을 Python 객체로 변환

In [ ]:
import json

# [기본] Python 딕셔너리를 JSON 파일로 저장
user_data = {
    "name": "홍길동",
    "age": 30,
    "city": "서울",
    "skills": ["Python", "SQL", "Excel"],
    "is_active": True
}

# JSON 파일로 저장
with open('user.json', 'w', encoding='utf-8') as f:
    json.dump(user_data, f, ensure_ascii=False, indent=2)
    # ensure_ascii=False: 한글 그대로 저장
    # indent=2: 들여쓰기로 가독성 향상

print("JSON 파일 저장 완료!")

In [ ]:
# JSON 파일 읽기
with open('user.json', 'r', encoding='utf-8') as f:
    loaded_data = json.load(f)

print(type(loaded_data))  # <class 'dict'>
print(loaded_data)
print(f"이름: {loaded_data['name']}")
print(f"스킬: {', '.join(loaded_data['skills'])}")

### 3.2 JSON 문자열 다루기 (dumps/loads)

In [ ]:
# [실습 1] Python 객체 → JSON 문자열
product = {
    "id": "P001",
    "name": "노트북",
    "price": 1200000
}

json_string = json.dumps(product, ensure_ascii=False, indent=2)
print(type(json_string))  # <class 'str'>
print(json_string)

In [ ]:
# [실습 2] JSON 문자열 → Python 객체
api_response = '{"status": "success", "data": {"user_id": 123, "username": "john"}}'

parsed = json.loads(api_response)
print(type(parsed))  # <class 'dict'>
print(f"상태: {parsed['status']}")
print(f"사용자 ID: {parsed['data']['user_id']}")

### 3.3 실무 예제: API 응답 처리

In [ ]:
# [실무 예제] 여러 사용자 데이터 저장
users = [
    {"id": 1, "name": "김철수", "department": "개발팀"},
    {"id": 2, "name": "이영희", "department": "마케팅팀"},
    {"id": 3, "name": "박민수", "department": "데이터팀"}
]

# 리스트를 JSON 파일로 저장
with open('users.json', 'w', encoding='utf-8') as f:
    json.dump(users, f, ensure_ascii=False, indent=2)

# 다시 읽어오기
with open('users.json', 'r', encoding='utf-8') as f:
    loaded_users = json.load(f)

# 데이터 분석팀 직원만 필터링
data_team = [u for u in loaded_users if u['department'] == '데이터팀']
print(f"데이터팀 인원: {data_team}")

---

## 4. [Pickle] Python 객체 직렬화

### 4.1 pickle이란?

**pickle**: Python 전용 직렬화 모듈
* **장점**: 거의 모든 Python 객체를 저장할 수 있음 (함수, 클래스, NumPy 배열 등)
* **단점**: Python에서만 읽을 수 있음 (다른 언어와 호환 불가)
* **보안**: 신뢰할 수 없는 pickle 파일을 로드하면 위험! (악성 코드 실행 가능)

**JSON vs Pickle**:
* JSON: 범용 포맷, 사람이 읽을 수 있음, 단순한 데이터 구조
* Pickle: Python 전용, 바이너리 포맷, 복잡한 객체도 저장 가능

In [ ]:
import pickle

# [기본] pickle로 저장하기
data = {
    "name": "분석 결과",
    "results": [1, 2, 3, 4, 5],
    "metadata": {"version": "1.0", "date": "2025-01-01"}
}

# 바이너리 쓰기 모드 ('wb')
with open('data.pkl', 'wb') as f:
    pickle.dump(data, f)

print("Pickle 파일 저장 완료!")

In [ ]:
# pickle 파일 읽기
with open('data.pkl', 'rb') as f:
    loaded_data = pickle.load(f)

print(loaded_data)
print(type(loaded_data))

### 4.2 실무 활용: 머신러닝 모델 저장

Pickle은 주로 다음과 같은 경우에 사용됩니다:
* 학습된 머신러닝 모델 저장
* 중간 분석 결과 캐싱
* 복잡한 Python 객체 저장 (NumPy 배열, Pandas DataFrame 등)

In [ ]:
# [실무 예제] 분석 결과와 설정을 함께 저장
analysis_state = {
    "config": {
        "threshold": 0.85,
        "model_type": "random_forest"
    },
    "results": {
        "accuracy": 0.92,
        "predictions": [0, 1, 1, 0, 1]
    },
    "timestamp": "2025-01-01 10:00:00"
}

# 저장
with open('analysis_state.pkl', 'wb') as f:
    pickle.dump(analysis_state, f)

# 나중에 불러오기
with open('analysis_state.pkl', 'rb') as f:
    restored_state = pickle.load(f)

print(f"모델 정확도: {restored_state['results']['accuracy']}")

---

## 5. 3교시 실전 퀴즈 (Quizzes)

지금까지 배운 파일 I/O, Context Manager, JSON, Pickle을 모두 활용해 보세요.

### Q1. [파일 I/O] 로그 파일 읽기 (난이도: ⭐⭐)

다음과 같은 로그 파일이 있습니다. (먼저 파일을 생성하세요)

```
2025-01-01 ERROR Connection failed
2025-01-01 INFO User login
2025-01-01 ERROR Database timeout
2025-01-01 WARN Low memory
```

1. 위 내용으로 `server.log` 파일을 생성하세요.
2. 파일을 읽어서 `"ERROR"`가 포함된 줄만 출력하세요.
3. 총 몇 개의 에러가 있는지 세어 출력하세요.

In [ ]:
# [Q1 풀이 작성]
# 1. 로그 파일 생성
    log_content = """ 2025-01-01 ERROR Connection failed
2025-01-01 INFO User login
2025-01-01 ERROR Database timeout
2025-01-01 WARN Low memory """

# 2. ERROR 로그만 출력
print("===ERROR 로그===")
error_count = 0

with open('server.log', 'w', encoding='utf-8') as f:
    for line in f:
        if 'ERROR' in line:
            print(line.strip())
            error_count += 1

# 3. 에러 개수 출력
print(f"\n총 에러 개수: {error_count}개")

### Q2. [Context Manager] 안전한 파일 쓰기 (난이도: ⭐⭐)

`함수 작성:`
* `safe_write_lines(filename, lines)` 함수를 작성하세요.
* `lines`는 문자열 리스트입니다.
* `with` 문을 사용하여 파일에 각 줄을 안전하게 작성합니다.
* 각 줄 끝에 줄바꿈(`\n`)을 추가합니다.

#### 예시
```python
safe_write_lines('output.txt', ['첫 번째 줄', '두 번째 줄', '세 번째 줄'])
# output.txt 파일이 생성되고 3줄이 작성됨
```

In [ ]:
# [Q2 풀이 작성]
def safe_write_lines(filename, lines):
    """
    with 문을 사용하여 안전하게 파일에 줄들을 작성
    
    Args:
        filename (str): 저장할 파일명
        lines (list): 작성할 문자열 리스트
    """
    # Your code here
    with open(filename, 'w', encoding='utf-8') as f:
        for line in lines:
            f.write(line + '\n')
    pass

# 테스트 코드
safe_write_lines('test_output.txt', ['Line 1', 'Line 2', 'Line 3'])
print("파일 작성 완료!")
# safe_write_lines('test_output.txt', ['Line 1', 'Line 2', 'Line 3'])

### Q3. [JSON] 사용자 데이터 관리 (난이도: ⭐⭐⭐)

다음 사용자 데이터를 JSON 파일로 저장하고 다시 읽어옵니다.

```python
user = {
    "user_id": "U001",
    "name": "김데이터",
    "email": "data@example.com",
    "preferences": {
        "language": "ko",
        "notifications": True
    }
}
```

1. 위 데이터를 `user_profile.json` 파일로 저장하세요. (한글이 깨지지 않도록)
2. 파일을 읽어서 사용자의 이름과 언어 설정을 출력하세요.
3. `preferences`에 `"theme": "dark"`를 추가하고 파일을 다시 저장하세요.

In [ ]:
user = {
    "user_id": "U001",
    "name": "김데이터",
    "email": "data@example.com",
    "preferences": {
        "language": "ko",
        "notifications": True
    }
}

# [Q3 풀이 작성]
# 1. JSON 파일로 저장
import json
    with open('user_info.json', 'w', encoding='utf-8') as f:
        json.dump(user, f, ensure_ascii=False, indent=4)
# 2. 파일 읽고 정보 출력
    with open('user_info.json', 'r', encoding='utf-8') as f:
        loaded_user = json.load(f)

    print(f"이름: {loaded_user['name']}")
    print(f"언어 설정: {loaded_user['preference']['language']}")
# 3. theme 추가 후 재저장
    loaded_user['preference']['theme'] = 'dark'

    with open ('user_info.json', 'r', encoding='utf-8') as f:
        json.dump(loaded_user, f, ensure_ascii=False, indent=4)

# 4. 확인
    with open(loaded_user.json, 'r', encoding='utf-8') as f:
        print('\n최종프로필')
        print(json.dump(json.load(f), ensure_ascii=False, indent=4))

### Q4. [JSON] 데이터 변환 및 필터링 (난이도: ⭐⭐⭐)

`함수 작성:`
* `filter_products_by_price(json_file, min_price)` 함수를 작성하세요.
* JSON 파일에서 제품 리스트를 읽어옵니다.
* `price`가 `min_price` 이상인 제품만 필터링하여 **새로운 리스트**를 반환합니다.

먼저 다음 데이터로 `products.json` 파일을 생성하세요:
```python
[
  {"name": "Mouse", "price": 15000},
  {"name": "Keyboard", "price": 45000},
  {"name": "Monitor", "price": 250000}
]
```

#### 예시
```python
result = filter_products_by_price('products.json', 40000)
print(result)  # [{'name': 'Keyboard', 'price': 45000}, {'name': 'Monitor', 'price': 250000}]
```

In [ ]:
# 먼저 테스트용 JSON 파일 생성
import json

products = [
    {"name": "Mouse", "price": 15000},
    {"name": "Keyboard", "price": 45000},
    {"name": "Monitor", "price": 250000}
]

with open('prodjct.json', 'w', encoding='utf-8') as f:
    json.dump(products, f, ensure_ascii=False, indent=4)

# [Q4 풀이 작성]
def filter_products_by_price(json_file, min_price):
    """
    JSON 파일에서 최소 가격 이상의 제품만 필터링
    
    Args:
        json_file (str): JSON 파일 경로
        min_price (int): 최소 가격
    
    Returns:
        list: 필터링된 제품 리스트
    """
    # Your code here
    with open(json_file, 'w', encoding='etf-8') as f:
        prpoducts = json.load(f)
    
    filtered = [p for p in products if p['price']>=min_price]
    return filtered 

# 테스트 코드
# result = filter_products_by_price('products.json', 40000)
# print(result)

### Q5. [JSON] 여러 파일 병합 (난이도: ⭐⭐⭐⭐)

여러 JSON 파일에 나누어진 데이터를 하나로 병합합니다.

`함수 작성:`
* `merge_json_files(file_list, output_file)` 함수를 작성하세요.
* `file_list`의 각 JSON 파일은 딕셔너리 **리스트**를 포함합니다.
* 모든 파일의 데이터를 하나의 리스트로 병합하여 `output_file`에 저장합니다.

테스트를 위해 2개의 JSON 파일을 먼저 생성하세요:
```python
# data1.json
[{"id": 1, "name": "A"}, {"id": 2, "name": "B"}]

# data2.json
[{"id": 3, "name": "C"}, {"id": 4, "name": "D"}]
```

#### 예시
```python
merge_json_files(['data1.json', 'data2.json'], 'merged.json')
# merged.json에 4개 항목이 모두 저장됨
```

In [ ]:
# 테스트용 파일 생성
data1 = [{"id": 1, "name": "A"}, {"id": 2, "name": "B"}]
data2 = [{"id": 3, "name": "C"}, {"id": 4, "name": "D"}]

with open('data1.json', 'w', encoding='utf-8') as f:
    json.dump(data1, f)

with open('data2.json', 'w', encoding='utf-8') as f:
    json.dump(data2, f)

# [Q5 풀이 작성]
def merge_json_files(file_list, output_file):
    """
    여러 JSON 파일의 데이터를 하나로 병합
    
    Args:
        file_list (list): JSON 파일 경로 리스트
        output_file (str): 출력 파일 경로
    """
    # Your code here
    pass

# 테스트 코드
# merge_json_files(['data1.json', 'data2.json'], 'merged.json')
# with open('merged.json', 'r') as f:
#     print(json.load(f))

### Q6. [종합] CSV to JSON 변환기 (난이도: ⭐⭐⭐⭐)

CSV 형식의 텍스트 파일을 읽어서 JSON 파일로 변환합니다.

`함수 작성:`
* `csv_to_json(csv_file, json_file)` 함수를 작성하세요.
* CSV 파일의 첫 줄은 헤더(컬럼명)입니다.
* 나머지 줄은 데이터입니다.
* 각 행을 딕셔너리로 변환하여 리스트에 담고 JSON 파일로 저장합니다.

먼저 다음 내용으로 `sales.csv` 파일을 생성하세요:
```
product,quantity,price
Apple,100,1000
Banana,200,500
Orange,150,800
```

#### 예시
```python
csv_to_json('sales.csv', 'sales.json')
# sales.json 내용:
# [
#   {"product": "Apple", "quantity": "100", "price": "1000"},
#   {"product": "Banana", "quantity": "200", "price": "500"},
#   {"product": "Orange", "quantity": "150", "price": "800"}
# ]
```

In [ ]:
# 테스트용 CSV 파일 생성
csv_content = """product,quantity,price
Apple,100,1000
Banana,200,500
Orange,150,800"""

with open('sales.csv', 'w', encoding='utf-8') as f:
    f.write(csv_content)

# [Q6 풀이 작성]
def csv_to_json(csv_file, json_file):
    """
    CSV 파일을 JSON 파일로 변환
    
    Args:
        csv_file (str): 입력 CSV 파일 경로
        json_file (str): 출력 JSON 파일 경로
    """
    # Your code here
    pass

# 테스트 코드
# csv_to_json('sales.csv', 'sales.json')
# with open('sales.json', 'r', encoding='utf-8') as f:
#     print(json.load(f))

### Q7. [종합/심화] 데이터 파이프라인 구축 (난이도: ⭐⭐⭐⭐⭐)

여러 단계를 거쳐 데이터를 처리하는 파이프라인을 구축합니다.

**시나리오**: 여러 부서의 매출 데이터(CSV)를 수집하여 통합 리포트(JSON)를 생성합니다.

다음 3개의 CSV 파일이 있습니다:
```
# dept_a.csv
date,amount
2025-01-01,100000
2025-01-02,150000

# dept_b.csv
date,amount
2025-01-01,200000
2025-01-02,180000

# dept_c.csv
date,amount
2025-01-01,120000
2025-01-02,140000
```

`작업:`
1. 각 CSV 파일을 읽어 파싱합니다.
2. 날짜별로 모든 부서의 매출을 합산합니다.
3. 결과를 다음 형식의 JSON 파일(`report.json`)로 저장합니다:
```json
{
  "2025-01-01": {"total": 420000, "departments": 3},
  "2025-01-02": {"total": 470000, "departments": 3}
}
```
4. 전체 기간 총 매출을 계산하여 출력합니다.

In [ ]:
# 테스트용 CSV 파일 생성
csv_files = {
    'dept_a.csv': "date,amount\n2025-01-01,100000\n2025-01-02,150000",
    'dept_b.csv': "date,amount\n2025-01-01,200000\n2025-01-02,180000",
    'dept_c.csv': "date,amount\n2025-01-01,120000\n2025-01-02,140000"
}

for filename, content in csv_files.items():
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(content)

# [Q7 풀이 작성]
# 1. CSV 파일들 읽기
    
# 2. 날짜별로 매출 합산

# 3. JSON 파일로 저장

# 4. 총 매출 계산 및 출력

---

## 🎯 학습 정리

### 핵심 개념 복습

| 개념 | 문법/함수 | 사용 예시 |
|------|----------|----------|
| 파일 열기 | `open(file, mode)` | `open('data.txt', 'r', encoding='utf-8')` |
| Context Manager | `with open(...) as f:` | 자동으로 파일 닫기 |
| 파일 읽기 | `.read()` / `.readlines()` | 전체 / 줄 단위 읽기 |
| JSON 저장 | `json.dump(obj, file)` | Python 객체 → JSON 파일 |
| JSON 읽기 | `json.load(file)` | JSON 파일 → Python 객체 |
| JSON 문자열 | `json.dumps()` / `json.loads()` | 문자열 변환 |
| Pickle 저장 | `pickle.dump(obj, file)` | Python 객체 직렬화 |

### 파일 모드 정리

| 모드 | 설명 | 파일 없으면 | 파일 있으면 |
|------|------|-----------|----------|
| `'r'` | 읽기 | 에러 | 읽기 |
| `'w'` | 쓰기 | 생성 | **덮어쓰기** |
| `'a'` | 추가 | 생성 | 끝에 추가 |
| `'x'` | 배타적 생성 | 생성 | 에러 |
| `'b'` | 바이너리 | (다른 모드와 조합) | - |

### 실무 Best Practices

1. **항상 `with` 문 사용**: 파일을 자동으로 닫아 리소스 누수 방지
2. **인코딩 명시**: `encoding='utf-8'`로 한글 깨짐 방지
3. **JSON vs Pickle**: 범용성이 필요하면 JSON, Python 전용이면 Pickle
4. **대용량 파일**: 한 줄씩 읽는 제너레이터 패턴 사용
5. **에러 처리**: `try-except`로 파일 없음 등의 예외 처리

### 다음 학습

- `csv` 모듈: CSV 파일 전용 처리
- `pathlib`: 경로 처리의 현대적 방법
- `pandas`: 데이터 분석을 위한 강력한 파일 I/O